In [15]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np
from torch.utils.data import TensorDataset

In [16]:
df = pd.read_parquet("small_tokenized_data.parquet")

In [17]:
#print(df.head())
IDS = df["input_ids"].tolist()
newIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    newIDS.append(each)

In [18]:
#get the attention mask
attn_mask_list = df["attention_mask"].tolist()
attn_mask=[]
for each in attn_mask_list:
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    attn_mask.append(each)

In [19]:
#get the star count AKA the y from half_data
stars = df["stars"].tolist()
starIDS=[]
for each in stars:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    starIDS.append(each)

In [20]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [21]:
#load data in batches for bert
bertData = TensorDataset(torch.tensor(newIDS), torch.tensor(attn_mask), torch.tensor(starIDS))
# set up a data loader to get batches from our dataset
bertLoader = torch.utils.data.DataLoader(bertData, batch_size = 16, shuffle = True)

In [22]:
batch = next(iter(bertLoader))
print(batch[0].shape)
print(batch[1].shape)
print(batch[2].shape)

torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16])


In [8]:
i = 0
for batch in bertLoader:
    y, pooled = bert(batch[0])
    torch.save(pooled, ("vector"+"%d"+".txt")%i)
        
    torch.save(batch[2], ("label"+"%d"+".txt")%i)

    print(i)
    i = i+1

0
1


KeyboardInterrupt: 

In [ ]:
#...final_pool = np.concatinate(list_pool/files)